In [1]:
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import os

def find_raw_folders(path):
    """
    This function recursively searches for all the folders named "raw"
    inside the given path and returns a list with their full paths.
    """
    i = 0
    raw_folders = []
    for item in os.listdir(path):
        item_path = os.path.join(path, item)
        if os.path.isdir(item_path):
            if item == "raw":
                raw_folders.append(item_path)
                print(item_path)
                i += 1
            else:
                sub_raw_folders = find_raw_folders(item_path)
                if len(sub_raw_folders) > 0:
                    raw_folders.extend(sub_raw_folders)
    return raw_folders


def get_num_files(path):
    # find all the "raw" folders
    raw_folder_paths = find_raw_folders(path)

    num_files = 0

    for raw_folder_path in raw_folder_paths:
        num_files += 1

    return num_files

In [3]:
import sys
import cv2
import dlib
import numpy as np

# Labels dict
behaviours = {'book': 0,
            'cell_phone': 1,
            'drink_water': 2,
            'speaking': 3,
            'stand_up': 4,
            'sit_down': 5,
            'face_up': 6,
            'face_down': 7,
            'face_left': 8,
            'face_right': 9,
            'face_upper_left': 10,
            'face_upper_right': 11,
            'face_lower_left': 12,
            'face_lower_right': 13,
            'hands_up': 14,
            'look_up': 15,
            'look_down': 16,
            'look_left': 17,
            'look_right': 18,
            'look_upper_left': 19,
            'look_upper_right': 20,
            'look_lower_left': 21,
            'look_lower_right': 22,
            'makefau1': 23,
            'makefau4': 24,
            'makefau5': 25,
            'makefau7': 26,
            'makefau9': 27,
            'makefau17': 28,
            'makefau23': 29,
            'makefau28': 30}

# video data input directory
data_dir = '/content/drive/My Drive/data/'

os.listdir(data_dir)



['.DS_Store', '20March23', '12April23', '02Mar23', '13March23', '22Feb23']

In [4]:
# get number of files, used for process checking
num_files = get_num_files(data_dir)

Streaming output truncated to the last 5000 lines.
/content/drive/My Drive/data/20March23/_63f8a51d30e2b7501f33e8ce@gmail.com/MakeFAU1/raw
/content/drive/My Drive/data/20March23/_63f8a51d30e2b7501f33e8ce@gmail.com/Look_Upper_Left/raw
/content/drive/My Drive/data/20March23/_63f8a51d30e2b7501f33e8ce@gmail.com/Look_Lower_Right/raw
/content/drive/My Drive/data/20March23/_63f8a51d30e2b7501f33e8ce@gmail.com/Look_Lower_Left/raw
/content/drive/My Drive/data/20March23/_63f8a51d30e2b7501f33e8ce@gmail.com/Look_Right/raw
/content/drive/My Drive/data/20March23/_63f8a51d30e2b7501f33e8ce@gmail.com/Look_Down/raw
/content/drive/My Drive/data/20March23/_63f8a51d30e2b7501f33e8ce@gmail.com/Hands_Up/raw
/content/drive/My Drive/data/20March23/_63f8a51d30e2b7501f33e8ce@gmail.com/Face_Upper_Right/raw
/content/drive/My Drive/data/20March23/_63f8a51d30e2b7501f33e8ce@gmail.com/Face_Upper_Left/raw
/content/drive/My Drive/data/20March23/_63f8a51d30e2b7501f33e8ce@gmail.com/Face_Right/raw
/content/drive/My Drive/dat

In [5]:
# find all the "raw" folders
raw_folder_paths = find_raw_folders(data_dir)

Streaming output truncated to the last 5000 lines.
/content/drive/My Drive/data/20March23/_63f8a51d30e2b7501f33e8ce@gmail.com/MakeFAU1/raw
/content/drive/My Drive/data/20March23/_63f8a51d30e2b7501f33e8ce@gmail.com/Look_Upper_Left/raw
/content/drive/My Drive/data/20March23/_63f8a51d30e2b7501f33e8ce@gmail.com/Look_Lower_Right/raw
/content/drive/My Drive/data/20March23/_63f8a51d30e2b7501f33e8ce@gmail.com/Look_Lower_Left/raw
/content/drive/My Drive/data/20March23/_63f8a51d30e2b7501f33e8ce@gmail.com/Look_Right/raw
/content/drive/My Drive/data/20March23/_63f8a51d30e2b7501f33e8ce@gmail.com/Look_Down/raw
/content/drive/My Drive/data/20March23/_63f8a51d30e2b7501f33e8ce@gmail.com/Hands_Up/raw
/content/drive/My Drive/data/20March23/_63f8a51d30e2b7501f33e8ce@gmail.com/Face_Upper_Right/raw
/content/drive/My Drive/data/20March23/_63f8a51d30e2b7501f33e8ce@gmail.com/Face_Upper_Left/raw
/content/drive/My Drive/data/20March23/_63f8a51d30e2b7501f33e8ce@gmail.com/Face_Right/raw
/content/drive/My Drive/dat

In [6]:
# Load the face detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/drive/My Drive/shape_predictor_68_face_landmarks.dat")

# specify output size
output_size = (64, 64)

In [7]:
# read the files inside all the "raw" folders
i = 0
for raw_folder_path in raw_folder_paths:
    for file in os.listdir(raw_folder_path):
        file_path = os.path.join(raw_folder_path, file)
        # get name of the video file
        video_name = str(file_path).split('/')[-1]

        # extract the behaviour name from the video
        temp = video_name[3:].lower()
        behaviour = "undefined"
        
        # to extinguish between cases like "makefau1" and "makefau17"
        # or cases like "face_up" and "face_upper_left"
        found = []
        for label in behaviours:
            if label in temp:
                found.append(label)
        if len(found) > 1:
            longest = max(found, key=len)
            behaviour = longest
        elif len(found) == 1:
            behaviour = found[0]
        else:
            print("unknwon behaviour")
            sys.exit(0)

        # Output path
        output_dir = '/content/drive/My Drive/tensors/' + behaviour + '_frames/'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Load the input video
        cap = cv2.VideoCapture(file_path)

        frames = []
        labels = []

        # Loop through each frame in the video
        while cap.isOpened():
            ret, frame = cap.read()
            if ret == False:
                break
        
            # Convert the frame to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Detect faces in the frame
            faces = detector(gray)

            if len(faces) != 1:
                output = None

            else:
                face = faces[0]
                landmarks = predictor(gray, face)
        
                # Extract the face part's image
                x1 = face.left()
                y1 = face.top()
                x2 = face.right()
                y2 = face.bottom()
                face_img = gray[y1:y2, x1:x2]

                if not face_img.size == 0:
                    # Resize the face image to the output size
                    face_img = cv2.resize(face_img, output_size)

                    # uncomment this section to see the grayscale images
                    # cv2.imshow('frame', face_img)
                    # key = cv2.waitKey(1)
                    
                    output = face_img
            
            if output is None:
                output = np.zeros_like(gray)
                output = cv2.resize(output, output_size)

            # Add frames and lables to list
            frames.append(output)
            labels.append(behaviours[behaviour])

        # Convert to np array
        frames = np.array(frames)
        labels = np.array(labels)

        # Get current number of files under the directory
        n = len(os.listdir(output_dir)) - 1

        # Save the frames and labels to a npz file, not text as to save disk space
        np.savez(os.path.join(output_dir, str(n + 1) + '.npz'), frames=frames, labels=labels)

        # Release the video capture object
        cap.release()

        # Close all windows
        cv2.destroyAllWindows()

    print("Current progress: {:.2%}".format(i / num_files))
    i += 1

Current progress: 0.00%
Current progress: 0.02%
Current progress: 0.04%
Current progress: 0.06%
Current progress: 0.08%
Current progress: 0.10%
Current progress: 0.12%
Current progress: 0.14%
Current progress: 0.16%
Current progress: 0.18%
Current progress: 0.20%
Current progress: 0.22%
Current progress: 0.24%
Current progress: 0.26%
Current progress: 0.28%
Current progress: 0.30%
Current progress: 0.32%
Current progress: 0.34%
Current progress: 0.36%
Current progress: 0.38%
Current progress: 0.40%
Current progress: 0.42%
Current progress: 0.44%
Current progress: 0.46%
Current progress: 0.48%
Current progress: 0.50%
Current progress: 0.52%
Current progress: 0.54%
Current progress: 0.56%
Current progress: 0.57%
Current progress: 0.59%
Current progress: 0.61%
Current progress: 0.63%
Current progress: 0.65%
Current progress: 0.67%
Current progress: 0.69%
Current progress: 0.71%
Current progress: 0.73%
Current progress: 0.75%
Current progress: 0.77%
Current progress: 0.79%
Current progress

KeyboardInterrupt: ignored